In [ ]:
%env LLM_API_KEY=sk-替换为自己的Qwen API Key
%env LLM_BASE_URL=https://dashscope.aliyuncs.com/compatible-mode/v1

In [2]:
import langchain, langchain_community, pypdf, sentence_transformers, chromadb, langchain_core

for module in (langchain, langchain_core, langchain_community, pypdf, sentence_transformers, chromadb):
    print(f"{module.__name__:<30}{module.__version__}")

/opt/anaconda3/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


langchain                     0.2.10
langchain_core                0.2.28
langchain_community           0.2.9
pypdf                         4.3.1
sentence_transformers         3.0.1
chromadb                      0.5.4


In [3]:
import os
import pandas as pd

In [4]:
expr_version = 'retrieval_v10_contextual_compression'

preprocess_output_dir = os.path.join(os.path.pardir, 'outputs', 'v1_20240713')
expr_dir = os.path.join(os.path.pardir, 'experiments', expr_version)

# 读取文档

In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(os.path.join(os.path.pardir, 'data', '2024全球经济金融展望报告.pdf'))
documents = loader.load()

qa_df = pd.read_excel(os.path.join(preprocess_output_dir, 'question_answer.xlsx'))

In [6]:
len(documents)

53

# 文档切分

In [7]:
from uuid import uuid4
import os
import pickle

from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents, filepath, chunk_size=400, chunk_overlap=40, seperators=['\n\n\n', '\n\n'], force_split=False):
    if os.path.exists(filepath) and not force_split:
        print('found cache, restoring...')
        return pickle.load(open(filepath, 'rb'))

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=seperators
    )
    split_docs = splitter.split_documents(documents)
    for chunk in split_docs:
        chunk.metadata['uuid'] = str(uuid4())

    pickle.dump(split_docs, open(filepath, 'wb'))

    return split_docs

In [8]:
splitted_docs = split_docs(documents, os.path.join(preprocess_output_dir, 'split_docs.pkl'), chunk_size=500, chunk_overlap=50)

found cache, restoring...


# 检索

添加一个函数便于打印检索信息

In [9]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i + 1}:" + (f"\nsimilarity: {d.state['query_similarity_score']:.4f}" if hasattr(d, 'state') else '') + "\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

## 准备基础检索器

In [10]:
import shutil
import torch

from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import Chroma

model_path = 'BAAI/bge-large-zh-v1.5'

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device: {device}')

def get_embeddings(model_path):
    embeddings = HuggingFaceBgeEmbeddings(
        model_name=model_path,
        model_kwargs={'device': device},
        encode_kwargs={'normalize_embeddings': True},
        query_instruction='为这个句子生成表示以用于检索相关文章：'
    )
    return embeddings

def get_vector_db(embeddings, docs, db_name):
    persist_directory = os.path.join(expr_dir, 'chroma', db_name)
    shutil.rmtree(persist_directory, ignore_errors=True)

    vector_db = Chroma.from_documents(
        splitted_docs,
        embedding=embeddings,
        persist_directory=persist_directory
    )
    return vector_db

device: cuda


In [11]:
embeddings = get_embeddings(model_path)
vector_db = get_vector_db(embeddings, splitted_docs, 'contextual_compression')

search_kwargs可以不指定，但本系列文章统一使用上下文长度为3进行对比，因此此处也设置这个参数

In [12]:
base_retriever = vector_db.as_retriever(search_kwargs={'k': 3})

In [13]:
pretty_print_docs(base_retriever.invoke('2023年全球经济增长的特点是什么？'))

Document 1:

全球经济金融展望报告
中国银行研究院 2 2024年
图1：全球GDP增速（%）
资料来源：IMF，中国银行研究院
分区域看，全球经济复苏不均衡，各国存在较大差异。发达经济体增速明
显放缓，预计2023年增速较2022年下降1个百分点。其中，欧元区和英国经
济增速大幅下降，美国表现好于其他发达经济体。2023年三季度，欧元区和英
国GDP环比增速均由之前的正增长转为负增长，分别下降0.1%和0.03%；美
国GDP环比增长折年率为4.9%，比二季度增速高2.8个百分点。新兴经济体增
速与2022年大致持平，预计2023年增速比2022年下降0.1个百分点。其中，
东南亚等出口型经济体增长承压，拉美、非洲等大宗商品出口国增速放缓，中
东欧国家经济增速加快（图2）。
----------------------------------------------------------------------------------------------------
Document 2:

全球经济金融展望报告
中国银行研究院 9 2024年
综合上述因素，“复苏+分化”将成为2024年全球经济主线逻辑，预计全
年经济增速在2.5%左右，增速比2023年低0.2个百分点。
（二）重点国别/地区经济形势研判
1.美国经济增速将回落，高利率对经济的滞后影响将逐步显现
2023年前三季度美国经济未受加息明显冲击，出现超预期增长。一、二季
度GDP环比增长折年率分别为2.2%和2.1%，处于潜在增速附近，三季度大幅
提高至4.9%，为2022年以来最好季度表现（图8）。其中，私人消费和政府财
政支出是支撑美国经济增长的重要驱动力，前三季度上述两分项对美国经济增
长的贡献率分别为64.4%和23.5%。后疫情时代消费者在服务领域的压抑需求
得到释放。服务支出成为消费者支出领域中最稳定和增幅最大的板块，前三季
度服务消费贡献了私人消费增长的58.7%。这主要得益于美国就业市场的稳健，
1-10月非农就业人数均值远高于历史水平，失业率平均值为3.6%，低于2022
年同期和国会预算办公室估算的非周期性失业率水平。美国劳动者实际收入保
持增长，个人实际可支配收入月同比增速均值达4.2%，家庭财务状况良好，家
庭财富对个人支出的正面效应高于历史水平。财政支出对经

## 使用`LLMChainExtractor`

In [14]:
from langchain_community.chat_models import ChatOllama
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

llm = ChatOllama(base_url='http://localhost:11434', model='qwen2:7b-instruct')
extractor_compressor = LLMChainExtractor.from_llm(llm)
extractor_compression_retriever = ContextualCompressionRetriever(
    base_compressor=extractor_compressor, base_retriever=base_retriever
)

In [15]:
pretty_print_docs(extractor_compression_retriever.invoke('2023年全球经济增长的特点是什么？'))

Document 1:

全球经济增长的特点如下：

- 全球经济复苏不均衡，各国存在较大差异。
- 发达经济体增速明显放缓，预计2023年增速较2022年下降1个百分点。其中，欧元区和英国经济增速大幅下降；美国表现好于其他发达经济体。
- 欧元区和英国在2023年三季度的GDP环比增速均由之前的正增长转为负增长。
- 美国GDP环比增长折年率为4.9%，比二季度增速高2.8个百分点。
- 新兴经济体增速与2022年大致持平，预计2023年增速比2022年下降0.1个百分点。其中，东南亚等出口型经济体增长承压；拉美、非洲等大宗商品出口国增速放缓；中东欧国家经济增速加快。

以上是根据给定的上下文提取出的相关信息，用于回答关于2023年全球经济增长特点的问题。
----------------------------------------------------------------------------------------------------
Document 2:

复苏+分化将成为2024年全球经济主线逻辑，预计全年经济增速在2.5%左右。美国经济增速将回落，高利率对经济的滞后影响将逐步显现。后疫情时代消费者在服务领域的压抑需求得到释放，服务支出成为消费者支出领域中最稳定和增幅最大的板块。美国就业市场稳健，非农就业人数均值远高于历史水平，失业率平均值为3.6%，个人实际可支配收入月同比增速均值达4.2%。财政支出对经济增长的带动作用上升。制造业投资在二季度之前保持稳定增长，很大程度上对冲了利率抬升对美国房地产投资的负面影响。净出口对美国经济的拖累效应减弱，2023年美元维持强势表现，前三季度货物进口支出下降6.0%，而全球需求疲软导致美国货物出口规模减少。
----------------------------------------------------------------------------------------------------
Document 3:

近期东亚、东南亚等主要经济体出口下行趋势收窄，贸易呈现企稳迹象。9-10月，越南出口结束连续10个月的负增长态势，同比分别增长5.0%和6.7%。10月，韩国出口同比增长5.1%，是自2022年10月以来首次正增长。

2024年预计全球经济复苏将依旧疲软，且分化可能加剧

## 使用`LLMChainFilter`

`LLMChainFilter`只是要保留哪个文档，过滤掉哪个文档，不会更改文档本身的内容

In [16]:
from langchain.retrievers.document_compressors import LLMChainFilter

_filter = LLMChainFilter.from_llm(llm)
filter_compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=base_retriever
)

In [17]:
pretty_print_docs(filter_compression_retriever.invoke('2023年全球经济增长的特点是什么？'))

Document 1:

全球经济金融展望报告
中国银行研究院 2 2024年
图1：全球GDP增速（%）
资料来源：IMF，中国银行研究院
分区域看，全球经济复苏不均衡，各国存在较大差异。发达经济体增速明
显放缓，预计2023年增速较2022年下降1个百分点。其中，欧元区和英国经
济增速大幅下降，美国表现好于其他发达经济体。2023年三季度，欧元区和英
国GDP环比增速均由之前的正增长转为负增长，分别下降0.1%和0.03%；美
国GDP环比增长折年率为4.9%，比二季度增速高2.8个百分点。新兴经济体增
速与2022年大致持平，预计2023年增速比2022年下降0.1个百分点。其中，
东南亚等出口型经济体增长承压，拉美、非洲等大宗商品出口国增速放缓，中
东欧国家经济增速加快（图2）。
----------------------------------------------------------------------------------------------------
Document 2:

全球经济金融展望报告
中国银行研究院 7 2024年
（图7）。10月，世贸组织将2023年全球货物贸易增速预测值下调0.9个百分
点至0.8%，2023年国际贸易增长或为近几年最低水平。但近期东亚、东南亚等
主要经济体出口下行趋势收窄，贸易呈现企稳迹象。9-10月，越南出口结束连
续10个月的负增长态势，同比分别增长5.0%和6.7%。10月，韩国出口同比增
长5.1%，是自2022年10月以来首次正增长。
图7：全球货物贸易量指数和货物贸易价格指数（2010年=100）
资料来源：荷兰经济分析局，中国银行研究院
展望2024年，预计全球经济复苏将依旧疲软，且分化可能加剧。
一是发达经济体面临的“遏通胀和稳增长”两难问题更加突出，货币政策
走势将分化。疫后服务业复苏加快、劳动力市场紧平衡、地缘冲突加剧等因素
导致发达经济体核心通胀黏性较强，通胀回落速度较慢。2023年10月，IMF
将2024年全球通胀预测值由7月的5.2%上调至5.8%，并预测到2025年前大
多数国家的通胀率都将高于央行目标。通胀反复风险使当前各主要央行对通胀
走势保持警惕，预计在一段时间内政策利率将继续维持在高位。但随着经济下
行压力加大，主要经济体央行货币政策走势将逐渐分化。截至1

## 使用`LLMListwiseRerank`

`LLMListwiseRerank`使用zero-shot的方式对文档进行reranking，它的作用与`LLMChainFilter`类似，但是更加鲁邦也会耗费更多资源。建议使用强大一些的模型

In [18]:
from langchain.retrievers.document_compressors import LLMListwiseRerank

# 此处官方文档中介绍需要强大一些的LLM
# https://python.langchain.com/docs/how_to/contextual_compression/

# from langchain_openai import ChatOpenAI
# gpt4o_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# _filter = LLMListwiseRerank.from_llm(gpt4o_llm, top_n=1)

# 本文为了行文统一，依然还是使用Ollama中的Qwen2-7B
# 此处需要用到函数调用功能，直接使用ChatOllama类的对象会报错，需要使用OllamaFunctions，但由于涉及到函数调用，实际使用时并不稳定

from langchain_experimental.llms.ollama_functions import OllamaFunctions
ollama_func_llm = OllamaFunctions(model='qwen2:7b-instruct', base_url='http://localhost:11434', format='json')

_filter = LLMListwiseRerank.from_llm(ollama_func_llm, top_n=1)
listwise_rerank_compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=base_retriever
)

In [19]:
retrieved_docs = listwise_rerank_compression_retriever.invoke('2023年全球经济增长的特点是什么？')

In [20]:
pretty_print_docs(retrieved_docs)

Document 1:

全球经济金融展望报告
中国银行研究院 2 2024年
图1：全球GDP增速（%）
资料来源：IMF，中国银行研究院
分区域看，全球经济复苏不均衡，各国存在较大差异。发达经济体增速明
显放缓，预计2023年增速较2022年下降1个百分点。其中，欧元区和英国经
济增速大幅下降，美国表现好于其他发达经济体。2023年三季度，欧元区和英
国GDP环比增速均由之前的正增长转为负增长，分别下降0.1%和0.03%；美
国GDP环比增长折年率为4.9%，比二季度增速高2.8个百分点。新兴经济体增
速与2022年大致持平，预计2023年增速比2022年下降0.1个百分点。其中，
东南亚等出口型经济体增长承压，拉美、非洲等大宗商品出口国增速放缓，中
东欧国家经济增速加快（图2）。


## 使用`EmbeddingFilter`

In [21]:
from langchain.retrievers.document_compressors import EmbeddingsFilter

embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.6)
embeddings_filter_compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter, base_retriever=base_retriever
)

In [22]:
retrieved_docs = embeddings_filter_compression_retriever.invoke('2023年全球经济增长的特点是什么？')

In [23]:
pretty_print_docs(retrieved_docs)

Document 1:
similarity: 0.6317

全球经济金融展望报告
中国银行研究院 2 2024年
图1：全球GDP增速（%）
资料来源：IMF，中国银行研究院
分区域看，全球经济复苏不均衡，各国存在较大差异。发达经济体增速明
显放缓，预计2023年增速较2022年下降1个百分点。其中，欧元区和英国经
济增速大幅下降，美国表现好于其他发达经济体。2023年三季度，欧元区和英
国GDP环比增速均由之前的正增长转为负增长，分别下降0.1%和0.03%；美
国GDP环比增长折年率为4.9%，比二季度增速高2.8个百分点。新兴经济体增
速与2022年大致持平，预计2023年增速比2022年下降0.1个百分点。其中，
东南亚等出口型经济体增长承压，拉美、非洲等大宗商品出口国增速放缓，中
东欧国家经济增速加快（图2）。
----------------------------------------------------------------------------------------------------
Document 2:
similarity: 0.6217

全球经济金融展望报告
中国银行研究院 9 2024年
综合上述因素，“复苏+分化”将成为2024年全球经济主线逻辑，预计全
年经济增速在2.5%左右，增速比2023年低0.2个百分点。
（二）重点国别/地区经济形势研判
1.美国经济增速将回落，高利率对经济的滞后影响将逐步显现
2023年前三季度美国经济未受加息明显冲击，出现超预期增长。一、二季
度GDP环比增长折年率分别为2.2%和2.1%，处于潜在增速附近，三季度大幅
提高至4.9%，为2022年以来最好季度表现（图8）。其中，私人消费和政府财
政支出是支撑美国经济增长的重要驱动力，前三季度上述两分项对美国经济增
长的贡献率分别为64.4%和23.5%。后疫情时代消费者在服务领域的压抑需求
得到释放。服务支出成为消费者支出领域中最稳定和增幅最大的板块，前三季
度服务消费贡献了私人消费增长的58.7%。这主要得益于美国就业市场的稳健，
1-10月非农就业人数均值远高于历史水平，失业率平均值为3.6%，低于2022
年同期和国会预算办公室估算的非周期性失业率水平。美国劳动者实际收入保
持增长，个人实际可支配收入月同比增速均值达4.2%

In [24]:
# pretty_print_docs(vector_db.as_retriever(search_type="similarity_score_threshold", search_kwargs={'k': 2, 'score_threshold': 0.2}).invoke('2023年全球经济增长的特点是什么？'))

# 预测

In [25]:
from langchain.llms import Ollama

ollama_llm = Ollama(
    model='qwen2:7b-instruct',
    base_url='http://localhost:11434'
)

In [26]:
def rag(llm, retriever, question):
    prompt_tmpl = """
你是一个金融分析师，擅长根据所获取的信息片段，对问题进行分析和推理。
你的任务是根据所获取的信息片段（<<<<context>>><<<</context>>>之间的内容）回答问题。
回答保持简洁，不必重复问题，不要添加描述性解释和与答案无关的任何内容。
已知信息：
<<<<context>>>
{{knowledge}}
<<<</context>>>

问题：{{question}}
请回答：
""".strip()

    chunks = retriever.invoke(question)
    prompt = prompt_tmpl.replace('{{knowledge}}', '\n\n'.join([doc.page_content for doc in chunks])).replace('{{question}}', question)

    return llm.invoke(prompt), chunks

In [27]:
print(rag(ollama_llm, extractor_compression_retriever, '2023年全球经济增长的特点是什么？')[0])

2023年全球经济增长的特点表现为"复苏+分化"的主线逻辑，预计全年经济增速为2.5%，比2022年降低0.2个百分点。发达经济体如欧元区和英国增长速度显著放缓，而美国表现相对较好。新兴经济体增长速度与前一年大致持平，但整体上仍面临一定压力。


In [33]:
from tqdm.auto import tqdm

prediction_df = qa_df[qa_df['dataset'] == 'test'][['uuid', 'question', 'qa_type', 'answer']].rename(columns={'answer': 'ref_answer'})

def predict(llm, retriever, prediction_df):
    prediction_df = prediction_df.copy()
    answer_dict = {}

    for idx, row in tqdm(prediction_df.iterrows(), total=len(prediction_df)):
        uuid = row['uuid']
        question = row['question']
        answer, chunks = rag(ollama_llm, retriever, question)

        answer_dict[question] = {
            'uuid': uuid,
            'ref_answer': row['ref_answer'],
            'gen_answer': answer,
            'chunks': chunks
        }
    prediction_df.loc[:, 'gen_answer'] = prediction_df['question'].apply(lambda q: answer_dict[q]['gen_answer'])
    prediction_df.loc[:, 'chunks'] = prediction_df['question'].apply(lambda q: answer_dict[q]['chunks'])

    return prediction_df

In [34]:
pred_dfs = {}

In [35]:
for key, retriever in zip(
    ['Vanilla Retriever', 'LLMChainExtractor', 'LLMChainFilter', 'EmbeddingsFilter'],
    [base_retriever, extractor_compression_retriever, filter_compression_retriever, embeddings_filter_compression_retriever]
):
    if key in pred_dfs:
        continue
    print(f'predicting with {key}:')
    pred_dfs[key] = predict(ollama_func_llm, retriever, prediction_df)

predicting with Vanilla Retriever:


  0%|          | 0/100 [00:00<?, ?it/s]

predicting with LLMChainExtractor:


  0%|          | 0/100 [00:00<?, ?it/s]

predicting with LLMChainFilter:


  0%|          | 0/100 [00:00<?, ?it/s]

predicting with EmbeddingsFilter:


  0%|          | 0/100 [00:00<?, ?it/s]

# 评估

In [41]:
from langchain_openai import ChatOpenAI
import time

judge_llm = ChatOpenAI(
    api_key=os.environ['LLM_API_KEY'],
    base_url=os.environ['LLM_BASE_URL'],
    model_name='qwen2-72b-instruct',
    temperature=0
)

def evaluate(judge_llm, prediction_df):
    """
    对预测结果进行打分
    :param prediction_df: 预测结果，需要包含问题，参考答案，生成的答案，列名分别为question, ref_answer, gen_answer
    :return 打分模型原始返回结果
    """
    prompt_tmpl = """
你是一个经济学博士，现在我有一系列问题，有一个助手已经对这些问题进行了回答，你需要参照参考答案，评价这个助手的回答是否正确，仅回复“是”或“否”即可，不要带其他描述性内容或无关信息。
问题：
<question>
{{question}}
</question>

参考答案：
<ref_answer>
{{ref_answer}}
</ref_answer>

助手回答：
<gen_answer>
{{gen_answer}}
</gen_answer>
请评价：
    """
    results = []

    for _, row in tqdm(prediction_df.iterrows(), total=len(prediction_df)):
        question = row['question']
        ref_answer = row['ref_answer']
        gen_answer = row['gen_answer']

        prompt = prompt_tmpl.replace('{{question}}', question).replace('{{ref_answer}}', str(ref_answer)).replace('{{gen_answer}}', gen_answer).strip()
        result = judge_llm.invoke(prompt).content
        results.append(result)

        time.sleep(1)
    return results

In [42]:
for key, pred_df in pred_dfs.items():
    pred_df['raw_score'] = evaluate(judge_llm, pred_df)
    pred_df['score'] = (pred_df['raw_score'] == '是').astype(int)
    print(f"{key} unique raw score: {pred_df['raw_score'].unique()}")

  0%|          | 0/100 [00:00<?, ?it/s]

Vanilla Retriever unique raw score: ['是' '否']


  0%|          | 0/100 [00:00<?, ?it/s]

LLMChainExtractor unique raw score: ['是' '否']


  0%|          | 0/100 [00:00<?, ?it/s]

LLMChainFilter unique raw score: ['是' '否']


  0%|          | 0/100 [00:00<?, ?it/s]

EmbeddingsFilter unique raw score: ['否' '是']


In [43]:
from IPython import display

In [44]:
for key, pred_df in pred_dfs.items():
    pred_df['score'] = (pred_df['raw_score'] == '是').astype(int)
    print(f"{key} accuracy: {pred_df['score'].mean()}")

    display.display_html(pred_df[['qa_type', 'score']].groupby('qa_type').mean().reset_index())

Vanilla Retriever accuracy: 0.7


,qa_type,score
0,detailed,0.688172
1,large_context,0.857143


LLMChainExtractor accuracy: 0.6


,qa_type,score
0,detailed,0.602151
1,large_context,0.571429


LLMChainFilter accuracy: 0.49


,qa_type,score
0,detailed,0.483871
1,large_context,0.571429


EmbeddingsFilter accuracy: 0.14


,qa_type,score
0,detailed,0.118280
1,large_context,0.428571


In [45]:
for key, pred_df in pred_dfs.items():
    pred_df.to_excel(os.path.join(expr_dir, f'{expr_version}_{key}_prediction.xlsx'), index=False)